# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [6]:
results = pd.read_csv('..\Output_Data\Cities.csv')
results

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mehrān,33.1222,46.1646,75.236,27,1,2.00,IR,1666753662
1,1,Vaini,-21.2000,-175.2000,75.362,64,75,3.09,TO,1666753665
2,2,Hithadhoo,-0.6000,73.0833,82.274,74,57,5.32,MV,1666753666
3,3,Kauhajoki,62.4333,22.1833,37.382,99,100,1.70,FI,1666753667
4,4,Najrān,17.4924,44.1277,61.124,27,0,0.17,SA,1666753669
...,...,...,...,...,...,...,...,...,...,...
574,580,Shenkursk,62.1091,42.8960,28.796,94,21,1.96,RU,1666754488
575,581,Vostok,46.4856,135.8833,48.632,29,14,1.82,RU,1666754489
576,582,Bonavista,48.6499,-53.1147,52.646,83,89,3.47,CA,1666754491
577,583,Fallon,39.4735,-118.7774,48.074,52,75,2.57,US,1666754492


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Load the cities
results = pd.read_csv('..\Output_Data\Cities.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)

In [8]:
locations = results[["Lat","Lng"]]
map_weight = results['Humidity']


In [9]:
fig = gmaps.figure(zoom_level=2, center=(35,-84))

heat_layer = gmaps.heatmap_layer(locations, weights = map_weight,
                                dissipating = False, max_intensity=10,
                                point_radius=1)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [10]:
hotel_df = results.loc[(results["Max Temp"] < 80) & (results["Max Temp"] > 70) \
                                    & (results["Wind Speed"] < 10) \
                                    & (results["Cloudiness"] < 40)].dropna()
hotel_df.head(5)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mehrān,33.1222,46.1646,75.236,27,1,2.00,IR,1666753662
18,18,Coahuayana Viejo,18.7333,-103.6833,76.964,92,39,0.68,MX,1666753691
43,43,Maceió,-9.6658,-35.7353,74.642,88,0,2.06,BR,1666753727
54,55,Carnarvon,-24.8667,113.6333,72.356,48,0,8.80,AU,1666753744
56,57,Geraldton,-28.7667,114.6000,72.014,43,3,7.20,AU,1666753579


In [ ]:
# ideal_temp = results.loc[(results['Max Temp'] > 289 ) & (results['Max Temp'] < 297)]
# ideal_cloud = ideal_temp.loc[results['Cloudiness'] < 50 ]
# ideal_humidity = ideal_temp.loc[results['Humidity'] < 30 ]
# ideal_city = ideal_humidity.loc[results['Wind Speed'] < 4.0]



### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
hotel_df['Hotel Name'] = ''

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_coordinates = hotel_df[["Lat","Lng"]]
target_radius = 5000
target_type = 'hotel'
params = {
    'location': hotel_coordinates,
    'radius': target_radius,
    'type': target_type,
    'key': g_key
}
rating = hotel_df['Humidity']


In [13]:
# Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Send a request
response = requests.get(base_url, params=params)

In [19]:
print(response)

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
fig2 = gmaps.figure(zoom_level=3,center=(35,-90))
heat_layer = gmaps.heatmap_layer(locations, weights = rating,
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1
                                )
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig2.add_layer(heat_layer)
fig2.add_layer(marker_layer)
# Display figure
fig2

Figure(layout=FigureLayout(height='420px'))